## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-10-18-52-12 +0000,bbc,Netanyahu defends Gaza plans as Israel heavily...,https://www.bbc.com/news/articles/c5yll33v9gwo...
1,2025-08-10-18-46-34 +0000,nyt,C.D.C. Workers Say Shooting Manifests Worst Fe...,https://www.nytimes.com/2025/08/09/us/politics...
2,2025-08-10-18-46-20 +0000,nypost,"1,600 illegal migrants — including MS-13 membe...",https://nypost.com/2025/08/10/us-news/1600-ill...
3,2025-08-10-18-38-23 +0000,nyt,Trump’s Cartel Order Revives ‘Bitter’ Memories...,https://www.nytimes.com/2025/08/10/us/trumps-c...
4,2025-08-10-18-38-20 +0000,nyt,Netanyahu Doubles Down on Gaza Offensive After...,https://www.nytimes.com/2025/08/10/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2431/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,26
63,putin,13
86,ukraine,13
72,new,11
2,gaza,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
57,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...,67
33,2025-08-10-16-46-36 +0000,nyt,Ukraine and Europe Project United Front Ahead ...,https://www.nytimes.com/2025/08/10/world/europ...,64
148,2025-08-09-23-34-20 +0000,nypost,White House weighs inviting Zelensky to Trump’...,https://nypost.com/2025/08/09/us-news/white-ho...,63
143,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...,63
70,2025-08-10-12-17-48 +0000,nyt,"With Ukraine in the Balance, Trump and Putin H...",https://www.nytimes.com/2025/08/10/world/europ...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
57,67,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...
9,42,2025-08-10-18-24-00 +0000,wsj,Israel’s New War Plan Includes Fighting in Gaz...,https://www.wsj.com/world/middle-east/israels-...
143,37,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
100,29,2025-08-10-10-00-00 +0000,latimes,"Arellano: When the going gets tough for L.A., ...",https://www.latimes.com/california/story/2025-...
158,26,2025-08-09-22-47-00 +0000,wsj,Adriana Kugler hasn’t said why she is leaving ...,https://www.wsj.com/economy/central-banking/su...
148,24,2025-08-09-23-34-20 +0000,nypost,White House weighs inviting Zelensky to Trump’...,https://nypost.com/2025/08/09/us-news/white-ho...
79,23,2025-08-10-10-00-00 +0000,latimes,"California took center stage in ICE raids, but...",https://www.latimes.com/california/story/2025-...
161,22,2025-08-09-21-47-06 +0000,nypost,"US, Qatari mediators try to shape last-minute ...",https://nypost.com/2025/08/09/world-news/us-qa...
168,20,2025-08-09-20-17-00 +0000,bbc,Atlanta police officer dies after shooting nea...,https://www.bbc.com/news/articles/cp3ekez0k2xo...
12,19,2025-08-10-18-18-00 +0000,nypost,Kin of tragic tot tossed into East River suing...,https://nypost.com/2025/08/10/us-news/kin-of-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
